### Multiclass Classification Model for Tripadvisor Hotel Review Star Ratings

Imported packages

In [65]:
import gradio as gr
import pickle
import numpy as np
from PIL import Image 
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from pytesseract import image_to_string  

Loaded pipeline files from Flask directory. 


In [66]:
rf_model_path = "C:/Users/rushej2/csc1052/a1/2024-csc-1052-rushej-2/src/flask/rf_classifiers.pkl"
nb_model_path = "C:/Users/rushej2/csc1052/a1/2024-csc-1052-rushej-2/src/flask/nb_classifiers.pkl"
lr_model_path = "C:/Users/rushej2/csc1052/a1/2024-csc-1052-rushej-2/src/flask/lr_classifiers.pkl"

with open(rf_model_path, 'rb') as f:
    rf_classifiers = pickle.load(f)

with open(nb_model_path, 'rb') as f:
    nb_classifiers = pickle.load(f)

with open(lr_model_path, 'rb') as f:
    lr_classifiers = pickle.load(f)

Created function to convert image to string.

In [67]:
def ocr_predict(image):
    text = image_to_string(image)
    return text

Created function to apply selected model to converted string.

In [68]:
def predict_star_rating(model_name, image):
    review = ocr_predict(image)
    
    if model_name == 'Random Forest':
        classifiers = rf_classifiers
    elif model_name == 'Naive Bayes':
        classifiers = nb_classifiers
    elif model_name == 'Logistic Regression':
        classifiers = lr_classifiers

    probabilities = np.zeros((1, 5))
    for star, classifier in classifiers.items():
        probs = classifier.predict_proba([review])[:, 1]  
        probabilities[:, star - 1] = probs

    return np.argmax(probabilities, axis=1)[0] + 1


Created Gradio interface to accept image, convert to text, and output model results in notebook environment.

In [69]:
def create_interface():
    model_options = ['Random Forest', 'Naive Bayes', 'Logistic Regression']
    
    gr.Interface(
        fn=predict_star_rating,
        inputs=[
            gr.Dropdown(choices=model_options, label="Select Model"),
            gr.Image(label="Upload an Image", type="pil")
        ],
        allow_flagging=False,
        outputs=gr.Textbox(label="Predicted Star Rating"),
        live=True,
        title="Hotel Review Star Rating Predictor",
        description="Upload an image (containing text), select a model (Random Forest, Naive Bayes, or Logistic Regression), and get the predicted star rating.",
        theme="compact"
    ).launch(share=True)

create_interface()


c:\Users\rushej2\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\blocks.py:982: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-674b6ed7-15f35905204806d250cffe21;4e37ce13-ff18-4e1a-be4b-80b055378499)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
c:\Users\rushej2\AppData\Local\Programs\Python\Python39\lib\site-packages\gradio\interface.py:390: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7872
Running on public URL: https://8e8f48992b1b606cb1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
